# Dataframe preparation

Once we have downloaded all the datasets we will be using in our project, the first thing we should do is preparing the dataframe we will be working on.

In [1]:
# Importing modules
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
rating_cols = ['STATE_CODE_001','STRUCTURE_NUMBER_008','DECK_COND_058','SUPERSTRUCTURE_COND_059','SUBSTRUCTURE_COND_060'] 

df17 = pd.read_csv('./data/2017hwybronlyonefile.zip',compression = 'zip',sep = ',',encoding='latin-1',usecols = rating_cols)

df17.head()

In [4]:
# Function for new variable: Total Rating (TR)

def TotalRating(row):
    dr = row['DECK_COND_058']
    supr = row['SUPERSTRUCTURE_COND_059']
    subr = row['SUBSTRUCTURE_COND_060']
    minval = min(dr, supr, subr)
    maxval = max(dr, supr, subr)
   
    if minval <= 4:
        rating = minval
        
    elif minval >= 8:
        rating = (dr+supr+subr)/3
        
    else:
        medval = dr+subr+supr-minval-maxval
        rating = 0.5*minval+0.2*maxval+0.3*medval

    return rating


df17 = df17[(df17['DECK_COND_058']!='N') & (df17['SUPERSTRUCTURE_COND_059']!='N') & (df17['SUBSTRUCTURE_COND_060']!='N') ]

# Pasamos los valores de rating a números
df17['DECK_COND_058'] = pd.to_numeric(df17['DECK_COND_058'])
df17['SUPERSTRUCTURE_COND_059'] = pd.to_numeric(df17['SUPERSTRUCTURE_COND_059'])
df17['SUBSTRUCTURE_COND_060'] = pd.to_numeric(df17['SUBSTRUCTURE_COND_060'])

# Quitamos los nulls
df17.dropna(inplace=True)

# Creamos la columna con el Total Rating para cada año
df17['TR17'] = df17.apply(lambda row: TotalRating(row),axis=1)

df17.head(10)

df17.drop(['DECK_COND_058','SUPERSTRUCTURE_COND_059','SUBSTRUCTURE_COND_060'], axis=1, inplace = True)

df17.head()

# Vamos a leer el dataset del 2018 para hacer el join
total18_fil = pd.read_csv('total18_fil.csv')

df18 = total18_fil
df18['TR18'] = df18.apply(lambda row: TotalRating(row),axis=1)

df18.shape

# Since we are looking for joining all the datasets with this one by using the Structure number, we should first check that there are no duplicates on it
df18['STRUCTURE_NUMBER_008'].duplicated().value_counts()

duplicateRowsDF = df18[df18.duplicated(['STRUCTURE_NUMBER_008'])]
duplicateRowsDF['STATE_CODE_001'].value_counts()

df17.shape

df2 = pd.merge(df18, df17, how= 'left', on = ['STATE_CODE_001','STRUCTURE_NUMBER_008'])

df2.shape

In [6]:
filenames = []
for year in range(2008,2018):
    year = str(year)
    path = "./data/"+year+"hwybronlyonefile.zip"
    filenames.append(path)

In [7]:
filenames

['./data/2008hwybronlyonefile.zip',
 './data/2009hwybronlyonefile.zip',
 './data/2010hwybronlyonefile.zip',
 './data/2011hwybronlyonefile.zip',
 './data/2012hwybronlyonefile.zip',
 './data/2013hwybronlyonefile.zip',
 './data/2014hwybronlyonefile.zip',
 './data/2015hwybronlyonefile.zip',
 './data/2016hwybronlyonefile.zip',
 './data/2017hwybronlyonefile.zip']

In [8]:
def createTR (filename,colname):
        
    # Create dataframes
    df = pd.read_csv(filename, compression = 'zip',sep = ',',encoding='latin-1',usecols = rating_cols)
    
    # Remove 'N'
    df = df[(df['DECK_COND_058']!='N') & (df['SUPERSTRUCTURE_COND_059']!='N') & (df['SUBSTRUCTURE_COND_060']!='N') ]
    
    # To numeric
    df['DECK_COND_058'] = pd.to_numeric(df['DECK_COND_058'])
    df['SUPERSTRUCTURE_COND_059'] = pd.to_numeric(df['SUPERSTRUCTURE_COND_059'])
    df['SUBSTRUCTURE_COND_060'] = pd.to_numeric(df['SUBSTRUCTURE_COND_060'])
    
    # Remove nulls
    df.dropna(inplace=True)
    
    # Creamos la columna con el Total Rating para cada año
    df[colname] = df.apply(lambda row: TotalRating(row),axis=1)
    
    return df
    

In [19]:
df17 = createTR('./data/2017hwybronlyonefile.zip','TR17')

/home/dsc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [20]:
df17.to_csv (r'./df17.csv', index = None, header=True)

In [21]:
df16 = createTR('./data/2016hwybronlyonefile.zip','TR16')

In [22]:
df16.to_csv (r'./df16.csv', index = None, header=True)

In [23]:
df15 = createTR('./data/2015hwybronlyonefile.zip','TR15')
df15.to_csv (r'./df15.csv', index = None, header=True)

In [24]:
df14 = createTR('./data/2014hwybronlyonefile.zip','TR14')
df14.to_csv (r'./df14.csv', index = None, header=True)

In [25]:
df13 = createTR('./data/2013hwybronlyonefile.zip','TR13')
df13.to_csv (r'./df13.csv', index = None, header=True)

In [26]:
df12 = createTR('./data/2012hwybronlyonefile.zip','TR12')
df12.to_csv (r'./df12.csv', index = None, header=True)

In [27]:
df11 = createTR('./data/2011hwybronlyonefile.zip','TR11')
df11.to_csv (r'./df11.csv', index = None, header=True)

In [28]:
df10 = createTR('./data/2010hwybronlyonefile.zip','TR10')
df10.to_csv (r'./df10.csv', index = None, header=True)

In [59]:
df09 = createTR('./data/2009hwybronlyonefile.zip','TR09')
df09.to_csv (r'./df09.csv', index = None, header=True)

/home/dsc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1,66,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [60]:
df08 = createTR('./data/2008hwybronlyonefile.zip','TR08')
df08.to_csv (r'./df08.csv', index = None, header=True)

# DESCARGAR LOS CSV A PARTIR DE AQUI

In [9]:
df17 = pd.read_csv('df17.csv')
df16 = pd.read_csv('df16.csv')
df15 = pd.read_csv('df15.csv')
df14 = pd.read_csv('df14.csv')
df13 = pd.read_csv('df13.csv')
df12 = pd.read_csv('df12.csv')
df11 = pd.read_csv('df11.csv')
df10 = pd.read_csv('df10.csv')
df09 = pd.read_csv('df09.csv')
df08 = pd.read_csv('df08.csv')

In [10]:
# Vamos a leer el dataset del 2018 para hacer el join
total18_fil = pd.read_csv('total18_fil.csv')

In [11]:
df18 = total18_fil
df18['TR18'] = df18.apply(lambda row: TotalRating(row),axis=1)

In [12]:
df18.shape

(168028, 26)

In [13]:
df_list = [df18,df17,df16,df15,df14,df13,df12,df11,df10,df09,df08] 

for df in df_list:
    df.drop(['DECK_COND_058','SUPERSTRUCTURE_COND_059','SUBSTRUCTURE_COND_060'], axis=1, inplace = True)

In [14]:
df_all = df18

for df in df_list[1:]:
    df_all = pd.merge(df_all, df, how='left', on=['STATE_CODE_001','STRUCTURE_NUMBER_008'])

In [15]:
df_all.head(10)

,STATE_CODE_001,STRUCTURE_NUMBER_008,LAT_016,LONG_017,TRAFFIC_LANES_ON_028A,MEDIAN_CODE_033,DEGREES_SKEW_034,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,MAIN_UNIT_SPANS_045,MAX_SPAN_LEN_MT_048,STRUCTURE_LEN_MT_049,DECK_WIDTH_MT_052,SURFACE_TYPE_108A,YEAR_BUILT_027,AGE,DESIGN_LOAD_031,TRUCK_ADT,MAINTENANCE_021,FUNCTIONAL_CLASS_026,WATERWAY_EVAL_071,YEAR_RECONSTRUCTED_106,TR18,TR17,TR16,TR15,TR14,TR13,TR12,TR11,TR10,TR09,TR08
0,1,000013771700120,34465400.0,87390000.0,2,0,0,steel,2,1,17.7,18.9,9.8,bituminous,1958.0,61.0,heavy,253.4,5.0,urban,none,0.0,6.5,6.5,7.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
1,1,000040,33304800.0,86480500.0,4,2,0,steel,2,1,8.2,8.2,44.5,bituminous,1910.0,109.0,other,130.0,4.0,urban,none,0.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,1,000050,34575357.0,86371644.0,2,0,20,steel,2,1,9.4,9.4,6.7,concrete,1910.0,109.0,light,2.0,2.0,rural,high,1995.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
3,1,000060,34101800.0,86504800.0,3,0,0,concrete,2,3,10.4,34.7,15.2,concrete,1994.0,25.0,heavy,7.6,4.0,urban,none,0.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
4,1,000083,34531863.0,86243520.0,1,0,20,steel,2,2,3.7,7.3,4.0,concrete,1914.0,105.0,heavy,0.0,2.0,rural,high,0.0,6.2,6.2,6.2,6.2,6.2,6.2,4.0,4.0,4.0,4.0,4.0
5,1,000085,32093843.0,86582940.0,1,0,0,steel,2,8,4.0,18.6,5.0,concrete,1915.0,104.0,light,0.0,2.0,rural,low,1996.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
6,1,000136,32450279.0,85333220.0,1,0,0,steel,2,5,6.0,30.3,4.8,concrete,1920.0,99.0,other,0.0,2.0,rural,high,0.0,4.0,4.0,4.0,5.2,5.2,5.2,5.2,5.2,6.0,6.0,6.0
7,1,000138,32502400.0,85461200.0,1,0,0,steel,2,1,12.2,12.2,5.2,concrete,1920.0,99.0,light,0.0,4.0,rural,high,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,1,000147,33020433.0,85310223.0,1,0,0,steel,2,1,6.1,7.0,4.9,bituminous,1920.0,99.0,other,0.0,2.0,rural,high,0.0,5.2,5.2,5.2,5.2,5.2,6.0,6.0,6.0,6.0,6.0,6.0
9,1,000163,32571214.0,85425920.0,1,0,0,steel,2,1,12.2,12.2,4.1,concrete,1920.0,99.0,other,0.0,2.0,rural,high,0.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5


In [16]:
df_all.shape

(169366, 33)

In [17]:
# La razón por la que hemos ampliado el dataset a todos los puentes es porque en Pennsylvania, 
# donde inicialmente lo ibamos a hacer, cuando hacemos el join nos quedamos con 5 puentes con rating
# válido de 2010 hacia atrás.

df_all[(df_all['STATE_CODE_001'] == 42)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3928 entries, 122487 to 126414
Data columns (total 33 columns):
STATE_CODE_001            3928 non-null int64
STRUCTURE_NUMBER_008      3928 non-null object
LAT_016                   3928 non-null float64
LONG_017                  3928 non-null float64
TRAFFIC_LANES_ON_028A     3928 non-null int64
MEDIAN_CODE_033           3928 non-null int64
DEGREES_SKEW_034          3928 non-null int64
STRUCTURE_KIND_043A       3928 non-null object
STRUCTURE_TYPE_043B       3928 non-null int64
MAIN_UNIT_SPANS_045       3928 non-null int64
MAX_SPAN_LEN_MT_048       3928 non-null float64
STRUCTURE_LEN_MT_049      3928 non-null float64
DECK_WIDTH_MT_052         3928 non-null float64
SURFACE_TYPE_108A         3928 non-null object
YEAR_BUILT_027            3928 non-null float64
AGE                       3928 non-null float64
DESIGN_LOAD_031           3928 non-null object
TRUCK_ADT                 3928 non-null float64
MAINTENANCE_021           3928 non-nul

In [18]:
# Si consideramos todo el dataset vamos perdiendo datos pero seguimos con un número considerable de entradas

df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169366 entries, 0 to 169365
Data columns (total 33 columns):
STATE_CODE_001            169366 non-null int64
STRUCTURE_NUMBER_008      169366 non-null object
LAT_016                   169366 non-null float64
LONG_017                  169366 non-null float64
TRAFFIC_LANES_ON_028A     169366 non-null int64
MEDIAN_CODE_033           169366 non-null int64
DEGREES_SKEW_034          169366 non-null int64
STRUCTURE_KIND_043A       169366 non-null object
STRUCTURE_TYPE_043B       169366 non-null int64
MAIN_UNIT_SPANS_045       169366 non-null int64
MAX_SPAN_LEN_MT_048       169366 non-null float64
STRUCTURE_LEN_MT_049      169366 non-null float64
DECK_WIDTH_MT_052         169366 non-null float64
SURFACE_TYPE_108A         169366 non-null object
YEAR_BUILT_027            169366 non-null float64
AGE                       169366 non-null float64
DESIGN_LOAD_031           169366 non-null object
TRUCK_ADT                 169366 non-null float64
MAINT

In [48]:
tr_cols = list(df_all.columns)[-11:][::-1]
tr_cols

['TR08',
 'TR09',
 'TR10',
 'TR11',
 'TR12',
 'TR13',
 'TR14',
 'TR15',
 'TR16',
 'TR17',
 'TR18']

# CAMBIAR LOS NAN POR EL VALOR DEL AÑO ANTERIOR

In [20]:
years_rated = len(df_list)
years_rated

11

In [37]:
# Drop all the rows where half of the ratings are missing

df_all.dropna(subset= tr_cols, thresh=(years_rated // 2 + 1), inplace=True)

In [38]:
df_all.shape

(149286, 33)

In [41]:
df_all['TR08'].isna().value_counts()

False    130064
True      19222
Name: TR08, dtype: int64

In [49]:
# For the missing values in the rating columns that are left, we will equal them to the rating of the previous year
# Therefore, we should have all the rating from the first year we are considering

df_all.dropna(subset = [tr_cols[1]], inplace= True)

In [60]:
df_all.shape

(127066, 34)

In [51]:
for n in range(len(tr_cols)-1):
    df_all[tr_cols[n+1]] = df_all.apply(lambda row: 
                                        row[tr_cols[n]] if np.isnan(row[tr_cols[n+1]]) else row[tr_cols[n+1]],
                                        axis=1)
    

In [52]:
df_all.sample(20)

,STATE_CODE_001,STRUCTURE_NUMBER_008,LAT_016,LONG_017,TRAFFIC_LANES_ON_028A,MEDIAN_CODE_033,DEGREES_SKEW_034,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,MAIN_UNIT_SPANS_045,MAX_SPAN_LEN_MT_048,STRUCTURE_LEN_MT_049,DECK_WIDTH_MT_052,SURFACE_TYPE_108A,YEAR_BUILT_027,AGE,DESIGN_LOAD_031,TRUCK_ADT,MAINTENANCE_021,FUNCTIONAL_CLASS_026,WATERWAY_EVAL_071,YEAR_RECONSTRUCTED_106,TR18,TR17,TR16,TR15,TR14,TR13,TR12,TR11,TR10,TR09,TR08
82332,31,C008401515,41490600.0,97135400.0,2,0,0,steel,2,1,14.9,15.2,6.2,concrete,1976.0,43.0,other,0.00,2.0,rural,low,0.0,5.7,5.7,5.7,5.700000,5.700000,5.700000,5.700000,5.700000,5.700000,5.700000,5.700000
73171,29,29798,38555260.0,92044660.0,2,0,30,steel,2,1,8.8,10.0,7.3,none,2002.0,17.0,other,5.00,2.0,rural,high,0.0,7.2,7.2,7.2,8.333333,8.333333,8.333333,8.666667,8.666667,8.666667,8.666667,8.666667
166058,55,B68009800000000,44151274.0,88505936.0,2,0,13,concrete,2,2,27.4,56.1,13.4,concrete,2003.0,16.0,heavy,0.00,1.0,urban,none,0.0,7.0,7.0,7.0,7.000000,7.200000,7.200000,7.200000,7.200000,7.500000,7.500000,7.500000
96580,36,000000005510450,42575547.0,76515473.0,2,0,0,steel,2,2,30.5,61.6,10.4,concrete,1999.0,20.0,heavy,30.00,5.0,rural,none,0.0,8.0,8.0,8.0,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,9.000000,9.000000
16245,10,1464 030,39191995.0,75363476.0,2,0,13,steel,2,2,32.0,65.1,14.2,concrete,1990.0,29.0,heavy,429.65,1.0,rural,none,0.0,6.4,6.2,6.2,6.200000,6.200000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
144855,48,150150001607168,29340442.0,98195936.0,2,0,0,concrete,2,2,24.4,112.8,14.3,concrete,1986.0,33.0,heavy,2894.40,1.0,urban,none,0.0,7.0,7.0,7.2,7.200000,7.200000,7.200000,7.200000,7.200000,7.200000,7.200000,7.200000
52188,23,5553,43432816.0,70180918.0,2,0,0,steel,2,3,18.3,51.5,10.2,bituminous,1961.0,58.0,light,182.35,1.0,urban,low,0.0,3.0,3.0,3.0,3.000000,4.000000,4.000000,5.700000,5.700000,5.700000,5.700000,5.700000
95099,36,000000003328970,43143078.0,78484012.0,2,0,15,steel,2,1,12.8,13.4,11.6,bituminous,1947.0,72.0,light,75.36,2.0,rural,high,0.0,5.5,5.5,5.5,5.500000,5.500000,5.500000,5.500000,5.500000,5.500000,5.500000,6.000000
79481,31,C001445210,42285400.0,97033600.0,1,0,0,steel,2,1,9.4,9.8,4.8,concrete,1935.0,84.0,other,0.00,2.0,rural,high,0.0,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
94161,36,000000003025130,41583287.0,74311721.0,2,0,10,steel,2,1,12.2,13.1,8.5,bituminous,2006.0,13.0,heavy,8.04,2.0,rural,high,0.0,7.2,7.2,7.2,7.200000,7.200000,7.200000,7.400000,7.400000,8.666667,8.666667,8.666667


In [56]:
# Function that creates the deterioration rate, which is the mean of the differences between ratings every year

def det_formula(row):
    
    m=0
    cont_rehab = 0

    for i in range(1,len(tr_cols)):
        dif = row[tr_cols[i-1]]-row[tr_cols[i]]
        
        if dif >= 0: 
            m += dif
            
        else:
            dif = 0
            cont_rehab+=1
            m += dif
            
    return m/(i-cont_rehab)

In [57]:
df_all['DETERIORATION_RATE'] = df_all.apply(lambda row: det_formula(row), axis=1)

In [59]:
# A deterioration rate of 0 seems unreasonable in a 10 years gap.
(df_all['DETERIORATION_RATE']==0).value_counts()

False    89446
True     37620
Name: DETERIORATION_RATE, dtype: int64

In [63]:
# There are 37620 bridges that seem to have no deterioration in 10 years.
# We will drop out those bridges, that might have had false input data from the inspection engineer
df_all = df_all.loc[df_all['DETERIORATION_RATE'] > 0]

In [65]:
# Final dataset
df_all.shape

(89446, 34)

In [66]:
# Save to csv

df_all.to_csv (r'./df_all.csv', index = None, header=True)